In [63]:
inner_contours = [[['VERTEX_POINT', -18, 5, 38, 'VERTEX_POINT', -5, 5, 38, 'LINE', 'contour_id', 0], ['VERTEX_POINT', -5, 5, 38, 'VERTEX_POINT', -5, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', -5, 5, 24, 'VERTEX_POINT', -18, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', -18, 5, 24, 'VERTEX_POINT', -18, 5, 38, 'LINE', 'contour_id', 0]], [['VERTEX_POINT', -51, 5, 0, 'VERTEX_POINT', -41, 5, 0, 'LINE', 'contour_id', 1], ['VERTEX_POINT', -41, 5, 0, 'VERTEX_POINT', -41, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', -41, 5, -6, 'VERTEX_POINT', -51, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', -51, 5, -6, 'VERTEX_POINT', -51, 5, 0, 'LINE', 'contour_id', 1]], [['VERTEX_POINT', 0, 0, -31, 'VERTEX_POINT', 0, 0, -19, 'CIRCLE', 0, 0, -25, 6.2, 'contour_id', 2]]]

outer_contours = [['VERTEX_POINT', 89, 0, 49, 'VERTEX_POINT', 89, 0, -49, 'LINE', 'dist', 98.0], ['VERTEX_POINT', -89, 0, 49, 'VERTEX_POINT', 89, 0, 49, 'LINE', 'dist', 178.0], ['VERTEX_POINT', -89, 0, -49, 'VERTEX_POINT', -89, 0, 49, 'LINE', 'dist', 98.0], ['VERTEX_POINT', 89, 0, -49, 'VERTEX_POINT', -89, 0, -49, 'LINE', 'dist', 178.0]]


In [64]:
import math

import copy
def get_contour_points(contour):
    points = []
    for entity in contour:
        p1 = (entity[1],entity[2],entity[3])
        p2 = (entity[5],entity[6],entity[7])
        points.append(p1)
        points.append(p2)
    return points
def get_center_of_geometery(points):
    sum_x = 0
    sum_y = 0
    sum_z = 0
    m = len(points)
    for point in points:
        sum_x+=point[0]
        sum_y+=point[1]
        sum_z+=point[2]
        
    centroid = (sum_x/m,sum_y/m,sum_z/m)
    return centroid
def add_centroid(inner_contours):
    for contour in inner_contours:
        contour_points = get_contour_points(contour)
        contour_centroid = get_center_of_geometery(contour_points)
        for entity in contour:
            entity.append("centroid") 
            entity.append(contour_centroid)        
    return inner_contours

def create_initial_groups(inner_contours):
    for contour in inner_contours:
        for entity in contour:
            entity.append("g") 
            entity.append(1)        
    return inner_contours

def calc_dist(p1,p2):
    d = math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2+(p1[2]-p2[2])**2)
    return d 
def far_point(contour):
    far_point = None
    far_dist = 0
    points = get_contour_points(contour)
    centroid = get_center_of_geometery(points)
    for p in points:
        d = math.sqrt((p[0]-centroid[0])**2+(p[1]-centroid[1])**2+(p[2]-centroid[2])**2)
        if far_dist<d:
            far_dist=d
            far_point=p

    return far_dist
    

def check_colliding_contours(contour1,contour2):
    constant = 25 #mm 
    e1  = contour1[0]
    e2 = contour2[0]
    
    if e1[8]=="LINE":
        c1 = e1[14]
    else:
        c1 = e1[18]
    
    f1 = far_point(contour1)
    l1 = f1 + constant
        
    if e2[8]=="LINE":
        c2 = e2[14]
    else:
        c2 = e2[18]
    
    f2 = far_point(contour2)
    l2 = f2 + constant

    d = calc_dist(c1,c2)
    
    if d> l1+l2:
        return False
    elif d<= l1+l2:
        return True
    
    
    
def get_groups(contour):
    if contour[0][8] =="LINE":
        g = contour[0][12]
    else:
        g = contour[0][16]

    return g

def get_contour_id(contour):
    if contour[0][8] =="LINE":
        c_id = contour[0][10]

    else:
        c_id = contour[0][14]
    
    return c_id
def create_groups(inner_contours):
    idx=0
    while idx <=10:
        i=0
        ids = []
        for c1 in inner_contours:
            for c2 in [inner_contours[-i] for i in range(len(inner_contours))]:
                if c1 == c2:
                    continue
                # check if they are in the same group first 
                g1 = get_groups(c1)
                c_id1 = get_contour_id(c1)
                g2 = get_groups(c2)
                c_id2 = get_contour_id(c2)

                if c_id1 in ids or c_id2 in ids:
                    continue                 
                
                print(f"c1:{c_id1}, c2:{c_id2} - g1:{g1}, g2:{g2}")
                
                if g1 == g2:            
                    collide_check = check_colliding_contours(c1,c2)
                    if collide_check:
                        print("contours collides")
                        # choose between either contours to shift
                        # priority to circles to be at early stages
                        
                        if c1[0][8] == 'CIRCLE' and c2[0][8]!="CIRCLE":
                            #shift c2
                            for entity in c2: 
                                entity[12]+=1
                                ids.append(c_id2) 
                                
                        elif c1[0][8] != 'CIRCLE' and c2[0][8]=="CIRCLE":
                            #shift c1 
                            for entity in c1: 
                                entity[12]+=1 
                                ids.append(c_id1)
                        else:
                            #shift any contours 
                            for entity in c1: 
                                if entity[8] =="LINE":
                                    entity[12]+=1
                                    ids.append(c_id1)

                                else:
                                    entity[16]+=1 
                                    ids.append(c_id1)

        idx+=1
    return inner_contours
def get_width(outer_contours):
    width = 0 
    for contour in outer_contours:
        w = contour[10]
        if w > width:
            width = w 
        
    return width 

# increase will be always in same dim. 
def adjust_contour_points(inner_contours):
    inner_contours = copy.deepcopy(inner_contours)
    groups = set()
    for contour in inner_contours:
        g = get_groups(contour)
        groups.add(g)
    
    thickness = 6
    delta_x = get_width(outer_contours) + thickness
    jdx = 0 
    while jdx < len(groups):
        if jdx ==0:
            jdx+=1
            continue
        
        for contour in inner_contours:
            g  = get_groups(contour)
            if g == jdx+1 :
                for entity in contour:
                    entity[1]+=delta_x 
                    entity[5]+=delta_x 
                    
                delta_x = delta_x * 2
    
        jdx+=1
    return inner_contours,delta_x
def transform_contours_to_original(inner_contours):
    contours = copy.deepcopy(inner_contours)
    new_contours = []
    for contour in contours:
        new_entities = []
        for entity in contour:
            if entity[8]=="LINE":
                entity= entity[:11]
                new_entities.append(entity)
            else:
                entity= entity[:15]
                new_entities.append(entity)

        new_contours.append(new_entities)
    return new_contours 
def adjust_outer_contour(outer_contours,delta_x):
    width= get_width(outer_contours)
    new_contours = [] 
    for contour in outer_contours:
        if contour[10] == width:
            contour[5]+=delta_x
        new_contours.append(contour)
    return new_contours

In [65]:
def process_outer_contours(inner_contours,outer_contours):
    print("contours before")
    print(inner_contours)
    inner_contours = create_initial_groups(inner_contours)
    inner_contours = add_centroid(inner_contours)
    inner_contours_m = create_groups(inner_contours)
    contours_after,delta_x = adjust_contour_points(inner_contours_m)
    new_contours = transform_contours_to_original(contours_after)
    processed_outer_contours= adjust_outer_contour(outer_contours,delta_x)

    print("contours before")
    print(new_contours)

    return new_contours,processed_outer_contours

In [66]:
# calculate centroid
# create groups
# adjust inner contours points
# retransform contours to original data
# process outer contours data


In [67]:
inner_contours, outer_contours = process_outer_contours(inner_contours,outer_contours)

contours before
[[['VERTEX_POINT', -18, 5, 38, 'VERTEX_POINT', -5, 5, 38, 'LINE', 'contour_id', 0], ['VERTEX_POINT', -5, 5, 38, 'VERTEX_POINT', -5, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', -5, 5, 24, 'VERTEX_POINT', -18, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', -18, 5, 24, 'VERTEX_POINT', -18, 5, 38, 'LINE', 'contour_id', 0]], [['VERTEX_POINT', -51, 5, 0, 'VERTEX_POINT', -41, 5, 0, 'LINE', 'contour_id', 1], ['VERTEX_POINT', -41, 5, 0, 'VERTEX_POINT', -41, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', -41, 5, -6, 'VERTEX_POINT', -51, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', -51, 5, -6, 'VERTEX_POINT', -51, 5, 0, 'LINE', 'contour_id', 1]], [['VERTEX_POINT', 0, 0, -31, 'VERTEX_POINT', 0, 0, -19, 'CIRCLE', 0, 0, -25, 6.2, 'contour_id', 2]]]
c1:0, c2:2 - g1:1, g2:1
contours collides
c1:1, c2:2 - g1:1, g2:1
contours collides
c1:0, c2:2 - g1:2, g2:1
c1:0, c2:1 - g1:2, g2:2
contours collides
c1:1, c2:2 - g1:2, g2:1
c1:2, c2:1 - g1:1, g2:2
c1:0, c2:2 - g1:3, g2:1


In [68]:
for contour in outer_contours:
    print(contour)

['VERTEX_POINT', 89, 0, 49, 'VERTEX_POINT', 89, 0, -49, 'LINE', 'dist', 98.0]
['VERTEX_POINT', -89, 0, 49, 'VERTEX_POINT', 825.0, 0, 49, 'LINE', 'dist', 178.0]
['VERTEX_POINT', -89, 0, -49, 'VERTEX_POINT', -89, 0, 49, 'LINE', 'dist', 98.0]
['VERTEX_POINT', 89, 0, -49, 'VERTEX_POINT', 647.0, 0, -49, 'LINE', 'dist', 178.0]


In [69]:
for contour in inner_contours:
    print(contour)

[['VERTEX_POINT', 350.0, 5, 38, 'VERTEX_POINT', 363.0, 5, 38, 'LINE', 'contour_id', 0], ['VERTEX_POINT', 363.0, 5, 38, 'VERTEX_POINT', 363.0, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', 363.0, 5, 24, 'VERTEX_POINT', 350.0, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', 350.0, 5, 24, 'VERTEX_POINT', 350.0, 5, 38, 'LINE', 'contour_id', 0]]
[['VERTEX_POINT', 133.0, 5, 0, 'VERTEX_POINT', 143.0, 5, 0, 'LINE', 'contour_id', 1], ['VERTEX_POINT', 143.0, 5, 0, 'VERTEX_POINT', 143.0, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', 143.0, 5, -6, 'VERTEX_POINT', 133.0, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', 133.0, 5, -6, 'VERTEX_POINT', 133.0, 5, 0, 'LINE', 'contour_id', 1]]
[['VERTEX_POINT', 0, 0, -31, 'VERTEX_POINT', 0, 0, -19, 'CIRCLE', 0, 0, -25, 6.2, 'contour_id', 2]]


In [70]:
for contour in inner_contours:
    print(contour)

[['VERTEX_POINT', 350.0, 5, 38, 'VERTEX_POINT', 363.0, 5, 38, 'LINE', 'contour_id', 0], ['VERTEX_POINT', 363.0, 5, 38, 'VERTEX_POINT', 363.0, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', 363.0, 5, 24, 'VERTEX_POINT', 350.0, 5, 24, 'LINE', 'contour_id', 0], ['VERTEX_POINT', 350.0, 5, 24, 'VERTEX_POINT', 350.0, 5, 38, 'LINE', 'contour_id', 0]]
[['VERTEX_POINT', 133.0, 5, 0, 'VERTEX_POINT', 143.0, 5, 0, 'LINE', 'contour_id', 1], ['VERTEX_POINT', 143.0, 5, 0, 'VERTEX_POINT', 143.0, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', 143.0, 5, -6, 'VERTEX_POINT', 133.0, 5, -6, 'LINE', 'contour_id', 1], ['VERTEX_POINT', 133.0, 5, -6, 'VERTEX_POINT', 133.0, 5, 0, 'LINE', 'contour_id', 1]]
[['VERTEX_POINT', 0, 0, -31, 'VERTEX_POINT', 0, 0, -19, 'CIRCLE', 0, 0, -25, 6.2, 'contour_id', 2]]


In [71]:
def find_lowest_x_points(points):
    # Initialize variables to store the lowest x-coordinates and corresponding points.
    lowest_x1 = float('inf')  # Initialize to positive infinity.
    lowest_x2 = float('inf')  # Initialize to positive infinity.
    point1 = None
    point2 = None
    
    # Loop through the points and update the lowest x-coordinates and corresponding points.
    for point in points:
        x = point[0]
        if x < lowest_x1:
            lowest_x2 = lowest_x1
            point2 = point1
            lowest_x1 = x
            point1 = point
        elif x < lowest_x2:
            lowest_x2 = x
            point2 = point
    
    return [point1, point2]

# Test the function with your given points
points = [(89, 0, 49), (-89, 0, -49), (-89, 0, 49), (89, 0, -49)]
lowest_x1, lowest_x2 = find_lowest_x_points(points)
print("Points with the lowest x-coordinates:")
print(lowest_x1)
print(lowest_x2)


Points with the lowest x-coordinates:
(-89, 0, -49)
(-89, 0, 49)
